## Interactive maps using geemap

In [1]:
import ee
import geemap.eefolium as geemap
import matplotlib.pyplot as plt

In [2]:
ee.Initialize()

#### Florida mangrove region

In [3]:
# Retrive dataset as a collection of images from Earth Engine Catalog
mangrove_images_landsat = ee.ImageCollection('LANDSAT/MANGROVE_FORESTS')

# Grab the first image from the collection (there's only one in the collection)
mangrove_images_landsat = mangrove_images_landsat.first()

# Specify visualization parameters
mangrovesVis = {
      min: 0,
      max: 1.0,
      'palette': ['d40115'],
    }

# Instantiate a map
mangrove_map = geemap.Map() 

# Add mangrove dataset as a leyer to the interactive map using the visualization parameters
mangrove_map.addLayer(mangrove_images_landsat, mangrovesVis, 'Mangroves')

# Center the map using longitude, latitude, zoom level
mangrove_map.setCenter(-81, 25, 9)

# Display the map
mangrove_map

In [4]:
# Get useful information about the dataset
geemap.image_props(mangrove_images_landsat).getInfo()

{'IMAGE_DATE': '2000-01-01',
 'NOMINAL_SCALE': 30.359861978395436,
 'system:asset_size': '41.133541 MB',
 'system:band_names': ['1'],
 'system:id': 'LANDSAT/MANGROVE_FORESTS/2000',
 'system:index': '2000',
 'system:time_end': '2001-01-01 00:00:00',
 'system:time_start': '2000-01-01 00:00:00',
 'system:version': 1506796895089836}

#### Suriname mangrove region

In [5]:
# define the point of interest
suriname_lonlat = [-53.94, 5.61]
suriname_point = ee.Geometry.Point(suriname_lonlat)

In [6]:
# Instantiate a map
suriname_map = geemap.Map() 
suriname_map.add_basemap('SATELLITE')

# Add mangrove dataset as a leyer to the interactive map using the visualization parameters
suriname_map.addLayer(mangrove_images_landsat, mangrovesVis, 'Mangroves')

# Center the map using longitude, latitude, zoom level
suriname_map.centerObject(suriname_point, 13)

# Display the map
suriname_map

#### Suriname area Landsat 8 satellite data

In [7]:
# select year
year = 2016

# Retrieve Landsat-8 SR dataset
suriname_image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(suriname_point) \
    .filterDate(f'{year}-01-01', f'{year}-12-31') \
    .select('B[1-7]') \
    .sort('CLOUD_COVER') \
    .first()
    

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

landsat = geemap.Map()
landsat.centerObject(suriname_point, 8)
landsat.addLayer(suriname_image, vis_params, "Landsat-8")
landsat

#### Image masking

In [8]:
mangrove_mask = suriname_image.updateMask(mangrove_images_landsat.eq(1))
non_mangrove_mask = suriname_image.updateMask(mangrove_mask.unmask().Not())

In [9]:
geemap.image_band_names(mangrove_mask).getInfo()

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']

#### Random sampling of points

In [10]:
mangrove_training_pts = mangrove_mask.sample(**{
    'region': mangrove_mask.geometry(),
    'scale': 30,
    'numPixels': 10000,
    'seed': 5343,
    'geometries': True
})

In [11]:
non_mangrove_training_pts = non_mangrove_mask.sample(**{
    'region': non_mangrove_mask.geometry(),
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True
})

In [12]:
mangrove_training_pts.size().getInfo(), non_mangrove_training_pts.size().getInfo()

(93, 4960)

In [13]:
training_map = geemap.Map()
training_map.setCenter(*suriname_lonlat, 13)

vis_params = {
    'min': 0,
    'max': 100,
    'bands': ['B4']
}

mangrove_color = 'eb0000'
non_mangrove_color = '1c5f2c'

legend_dict = {
    'Mangrove Point': mangrove_color,
    'Non-mangrove Point': non_mangrove_color,
    'Mangrove Area': 'd1def8',
    'mangrove mask': (0, 0, 0),
    'non mangrove mask': (128, 128, 128)
}

training_map.addLayer(mangrove_mask, {'min': 0, 'max': 10000, 'bands': ['B1']}, 'mangrove mask', True)
training_map.addLayer(mangrove_training_pts, {'color': mangrove_color}, 'Mangrove Sample')
training_map.addLayer(non_mangrove_mask, {}, 'non mangrove mask', True)
training_map.addLayer(non_mangrove_training_pts, {'color': non_mangrove_color}, 'non mangrove training', True)
training_map.add_legend(legend_dict=legend_dict)
training_map.centerObject(ee.Geometry.Point(-54.8451, 5.9453), 11)

training_map

#### dataframe from sampled points

In [14]:
from geemap import ee_to_geopandas

mangrove_gdf = ee_to_geopandas(mangrove_training_pts)
mangrove_gdf["lon"] = mangrove_gdf["geometry"].apply(lambda p: p.x)
mangrove_gdf["lat"] = mangrove_gdf["geometry"].apply(lambda p: p.y)
mangrove_gdf["label"] = 1 
mangrove_gdf = mangrove_gdf.drop("geometry", axis=1)
print(mangrove_gdf.head())

     B1    B2    B3    B4    B5    B6    B7        lon       lat  label
0   251   326   623   535  1919   970   478 -53.952683  5.733402      1
1  4354  4483  4714  4779  5898  4587  3714 -53.383394  5.559823      1
2  1229  1249  1519  1455  3279  1961  1454 -53.754688  5.683997      1
3   259   312   596   411  3049  1644   740 -54.781271  5.954568      1
4   210   279   540   395  2689  1241   510 -54.722145  5.978066      1


In [15]:
mangrove_gdf.shape

(93, 10)

In [16]:
non_mangrove_gdf = ee_to_geopandas(non_mangrove_training_pts)
non_mangrove_gdf["lon"] = non_mangrove_gdf["geometry"].apply(lambda p: p.x)
non_mangrove_gdf["lat"] = non_mangrove_gdf["geometry"].apply(lambda p: p.y)
non_mangrove_gdf["label"] = 0
non_mangrove_gdf = non_mangrove_gdf.drop("geometry", axis=1)
print(non_mangrove_gdf.head())

    B1   B2   B3   B4    B5    B6   B7        lon       lat  label
0  101  169  386  248  3101  1277  508 -54.234656  5.501284      0
1  356  398  337  285   296   292  268 -53.242286  5.944391      0
2  378  409  232   88    71    87   78 -54.694873  6.638615      0
3  251  273  241  203   219   229  204 -53.336881  6.381612      0
4  114  166  393  229  3106  1256  474 -54.762566  5.522397      0
